In [15]:
import numpy as np
import pickle
import pandas as pd

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
import os

from src.src.uw_spider.fd import FD_Network

BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"


In [16]:
# model_name = "Web_spirals_r56_s19"
model_name = "Web_constant_length"
ecc = 2
gap = 0
spider_root = r"C:\Users\thijs\Documents\GitHub\uw_spider"
spider_path = os.path.join(spider_root, 'data', 'networks', "Constraint", model_name, model_name + f"_ecc{ecc}_gap{gap}_constraint.json")

net = FD_Network.from_json(spider_path)

In [26]:
import matplotlib.pyplot as plt

tex_lines = [r"\begin{tikzpicture}"]

# Compute min and max force values without storing them in a list
f_min, f_max = float("inf"), float("-inf")
for key in net.edges:
    f = net.f[key]
    if f < f_min:
        f_min = f
    if f > f_max:
        f_max = f

cmap = plt.get_cmap("viridis")  # Choose a colormap

gamma = 3  # Adjust this value to control the sensitivity to smaller values

for key, edge in net.edges.items():
    v0, v1 = edge
    f = net.f[key]
    coor0, coor1 = net.vertex_xyz(v0), net.vertex_xyz(v1)

    # Normalize force
    norm_f = (f - f_min) / (f_max - f_min) if f_max > f_min else 0.5

    # Apply gamma correction
    norm_f = norm_f**gamma  

    # Rescale back to [0,1] using Min-Max scaling
    min_gamma = f_min**gamma if f_min > 0 else 0  # Avoid issues with negative forces
    max_gamma = f_max**gamma if f_max > 0 else 1

    norm_f = (norm_f - min_gamma) / (max_gamma - min_gamma) if max_gamma > min_gamma else 0.5


    # Get RGB color from colormap
    r, g, b, _ = cmap(norm_f)

    tex_lines.append(fr"""
    \definecolor{{edge{key}}}{{rgb}}{{{r:.3f},{g:.3f},{b:.3f}}}
    \draw[color=edge{key}] 
        ({coor0[0]/10:.3f}, {coor0[1]/10:.3f}) -- 
        ({coor1[0]/10:.3f}, {coor1[1]/10:.3f});
    """)

tex_lines.append(r"\end{tikzpicture}")

# Write the file efficiently
with open("tikz_image_web.tex", "w") as f:
    f.write("\n".join(tex_lines))

print("tikz_image_colormap.tex generated successfully.")

f_min*3, f_max*3

tikz_image_colormap.tex generated successfully.


(0.0006506855142312154, 0.5349489750920539)

In [18]:
print_order = ['Frame_ring_keys', 'Radials_keys + connections', 'Spiral_center_keys', 'Spiral_catching_keys + UTurn_keys']
section_names = ['Frame_ring', 'Spiral_center', 'Frame_sec', 'Radials_keys', 'Spiral_catching', 'UTurn_keys']
All_sections_nonuniform = [net.Frame_ring_keys, net.Spiral_center_keys, net.Frame_sec_keys, net.Radials_keys, net.Spiral_catching_keys, net.UTurn_keys]

import matplotlib.pyplot as plt
netf = Network_custom()
tex_lines = [r"\begin{tikzpicture}"]


for section in All_sections_nonuniform:
    counter = 0
    for key, edge_key in reversed(section.items()):
        if edge_key in section_names:
            continue
        counter += 1
        edge = net.edges[edge_key]
        v0, v1 = edge
        R = net.R[edge_key]
        theta = net.th[edge_key]

        if counter %2 == 0:
            direction = 1
        else:
            direction = -1

        coor0, coor1 = net.vertex_xyz(v0), net.vertex_xyz(v1)
        center, theta0, theta1 = netf._points_on_arc(coor0[:2], coor1[:2], R, print_parameters = True, dir=direction)
        if R/10 > 10000:
            tex_lines.append(fr"""
            \draw[line width=0.5pt, color=black]
            
                ({coor0[0]/10:.3f}, {coor0[1]/10:.3f}) -- 
                ({coor1[0]/10:.3f}, {coor1[1]/10:.3f});
            """)
        elif theta0 > theta1:
            tex_lines.append(fr"""
            \draw ({coor0[0]/10:.3f}, {coor0[1]/10:.3f}) arc ({theta0*180/np.pi:.3f}:{theta1*180/np.pi:.3f}:{R/10:.3f});""")
        else:
            tex_lines.append(fr"""
            \draw ({coor1[0]/10:.3f}, {coor1[1]/10:.3f}) arc ({theta1*180/np.pi:.3f}:{theta0*180/np.pi:.3f}:{R/10:.3f});""")

tex_lines.append(r"\end{tikzpicture}")

# Write the file efficiently
with open("tikz_image_web_arc.tex", "w") as f:
    f.write("\n".join(tex_lines))

print("tikz_image_colormap.tex generated successfully.")


tikz_image_colormap.tex generated successfully.


In [19]:
file_name_gcode = "Web_constant_length_ecc2_new.gcode"
gcode_path = os.path.join(spider_root, "data", "gcode", file_name_gcode)

vertices_g = netf.read_gcode(gcode_path, save_every_n=2)

In [20]:
tikz_text = netf.tikz_string_vertices(vertices_g, scale=10)
with open("tikz_image_web_arc2.tex", "w") as f:
    f.write("\n".join(tikz_text))

In [21]:
tikz_text

['\\begin{tikzpicture}',
 '\\draw[black] (2.000, 0.000) -- (9.000, 0.000);',
 '\\draw[black] (18.236, 14.439) -- (18.225, 14.522);',
 '\\draw[black] (18.225, 14.522) -- (18.216, 14.606);',
 '\\draw[black] (18.216, 14.606) -- (18.210, 14.689);',
 '\\draw[black] (18.210, 14.689) -- (18.206, 14.773);',
 '\\draw[black] (18.206, 14.773) -- (18.205, 14.857);',
 '\\draw[black] (18.205, 14.857) -- (18.206, 14.940);',
 '\\draw[black] (18.206, 14.940) -- (18.209, 15.024);',
 '\\draw[black] (18.209, 15.024) -- (18.215, 15.108);',
 '\\draw[black] (18.215, 15.108) -- (18.223, 15.191);',
 '\\draw[black] (18.223, 15.191) -- (18.234, 15.274);',
 '\\draw[black] (18.234, 15.274) -- (18.247, 15.357);',
 '\\draw[black] (18.247, 15.357) -- (18.270, 15.435);',
 '\\draw[black] (18.270, 15.435) -- (18.302, 15.509);',
 '\\draw[black] (18.302, 15.509) -- (18.333, 15.584);',
 '\\draw[black] (18.333, 15.584) -- (18.362, 15.659);',
 '\\draw[black] (18.362, 15.659) -- (18.390, 15.735);',
 '\\draw[black] (18.390, 15